## Demo: SingleRun with HasDelayPlugin
The basic steps to set up an OpenCLSim simulation are:
* Import libraries
* Initialise simpy environment
* Define object classes
* Create objects
  * Create sites
  * Create vessels
  * Create activities
* Register processes and run simpy

----

This example notebook shows how you can integrate percentual delays in your simulation. We demonstrate how you can use the HasDelayPlugin, and how you can give it a certain delay percentage.

#### 0. Import libraries

In [1]:
import datetime, time
import simpy

import shapely.geometry
import pandas as pd
import numpy as np

import openclsim.core as core
import openclsim.model as model
import openclsim.plot as plot
import openclsim.plugins as plugins
import opentnsim.core

# TO DO
# See chat for power to velocity
# insert acceleration with power2v function. Better implementation for calculate power
# One notebook for calculating velocity depending on a set power. From 0 to end. Need to know resistance over time for increasing velocity.
# Add resistance for seagoing
# Depth integration. Varying depth over the graph -> see M v koningsveld. Might connect with opentnsim. move function -> velocity will be connected together.

In [2]:
def calculate_power(V_0=10, h=100):
    
    
    # Define all parameters (HAM318):
    L=225
    B=20
    T=19
    nu=0.000001
    C_b=0.85
    rho=1025
    c_stern = 1
    one_k2=2.5
    g=9.81


    ###########################################################################################################################

    """1) Frictional resistance
            - 1st resistance component defined by Holtrop and Mennen (1982)
            - A modification to the original friction line is applied, based on literature of Zeng (2018), to account for shallow water effects """

    C_M = 1.006 - 0.0056 * C_b ** (-3.56)  # Midship section coefficient
    C_wp = (1 + 2 * C_b) / 3  # Waterplane coefficient
    C_p = C_b / C_M  # Prismatic coefficient

    delta = C_b * L * B * T  # Water displacement

    lcb = -13.5 + 19.4 * C_p  # longitudinal center of buoyancy
    L_R = L * (1 - C_p + (0.06 * C_p * lcb) / (
                4 * C_p - 1))  # parameter reflecting the length of the run

    A_T = 0.2 * B * T  # transverse area of the transom

    # Total wet area
    S_T = L * (2 * T + B) * np.sqrt(C_M) * (
                0.453 + 0.4425 * C_b - 0.2862 * C_M - 0.003467 * (
                    B / T) + 0.3696 * C_wp)  # + 2.38 * (A_BT / C_b)

    S_APP = 0.05 * S_T  # Wet area of appendages
    S_B = L * B  # Area of flat bottom

    D_s = 0.7 * T  # Diameter of the screw


    R_e = V_0 * L / nu  # Reynolds number
    D = h - T  # distance from bottom ship to the bottom of the fairway

    # Friction coefficient in deep water
    Cf_0 = 0.075 / ((np.log10(R_e) - 2) ** 2)

    # Friction coefficient proposed, taking into account shallow water effects
    Cf_proposed = (0.08169 / ((np.log10(R_e) - 1.717) ** 2)) * (
                1 + (0.003998 / (np.log10(R_e) - 4.393)) * (D / L) ** (-1.083))

    # 'a' is the coefficient needed to calculate the Katsui friction coefficient
    a = 0.042612 * np.log10(R_e) + 0.56725
    Cf_katsui = 0.0066577 / ((np.log10(R_e) - 4.3762) ** a)

    # The average velocity underneath the ship, taking into account the shallow water effect

    if h / T <= 4:
        V_B = 0.4277 * V_0 * np.exp((h / T) ** (-0.07625))
    else:
        V_B = V_0

    # cf_proposed cannot be applied directly, since a vessel also has non-horizontal wet surfaces that have to be taken
    # into account. Therefore, the following formula for the final friction coefficient 'C_f' is defined:
    C_f = Cf_0 + (Cf_proposed - Cf_katsui) * (S_B / S_T) * (V_B / V_0) ** 2

    # The total frictional resistance R_f [kN]:
    R_f = (C_f * 0.5 * rho * (V_0 ** 2) * S_T) / 1000

    ########################################################################################################################

    """2) Viscous resistance
    - 2nd resistance component defined by Holtrop and Mennen (1982)
    - Form factor (1 + k1) has to be multiplied by the frictional resistance R_f, to account for the effect of viscosity"""

    # c_14 accounts for the specific shape of the afterbody
    c_14 = 1 + 0.0011 * c_stern

    # the form factor (1+k1) describes the viscous resistance
    one_k1 = 0.93 + 0.487 * c_14 * ((B / L) ** 1.068) * ((T / L) ** 0.461) * (
                (L / L_R) ** 0.122) * (((L ** 3) / delta) ** 0.365) * (
                              (1 - C_p) ** (-0.604))

    ########################################################################################################################


    """3) Appendage resistance
    - 3rd resistance component defined by Holtrop and Mennen (1982)
    - Appendages (like a rudder, shafts, skeg) result in additional frictional resistance"""

    # Frictional resistance resulting from wetted area of appendages: R_APP [kN]
    R_APP = (0.5 * rho * (V_0 ** 2) * S_APP * one_k2 * C_f) / 1000

    #################################################################################################################


    """Intermediate calculation: Karpov
    - The Karpov method computes a velocity correction that accounts for limited water depth (corrected velocity V2)
    - V2 has to be implemented in the wave resistance and the residual resistance terms"""

    # The Froude number used in the Karpov method is the depth related froude number F_nh

    # The different alpha** curves are determined with a sixth power polynomial approximation in Excel
    # A distinction is made between different ranges of Froude numbers, because this resulted in a better approximation of the curve
    F_nh = V_0 / np.sqrt(g * h)

    if F_nh <= 0.4:

        if 0 <= h / T < 1.75:
            alpha_xx = (-4 * 10 ** (
                -12)) * F_nh ** 3 - 0.2143 * F_nh ** 2 - 0.0643 * F_nh + 0.9997
        if 1.75 <= h / T < 2.25:
            alpha_xx = -0.8333 * F_nh ** 3 + 0.25 * F_nh ** 2 - 0.0167 * F_nh + 1
        if 2.25 <= h / T < 2.75:
            alpha_xx = -1.25 * F_nh ** 4 + 0.5833 * F_nh ** 3 - 0.0375 * F_nh ** 2 - 0.0108 * F_nh + 1
        if h / T >= 2.75:
            alpha_xx = 1

    if F_nh > 0.4:
        if 0 <= h / T < 1.75:
            alpha_xx = -0.9274 * F_nh ** 6 + 9.5953 * F_nh ** 5 - 37.197 * F_nh ** 4 + 69.666 * F_nh ** 3 - 65.391 * F_nh ** 2 + 28.025 * F_nh - 3.4143
        if 1.75 <= h / T < 2.25:
            alpha_xx = 2.2152 * F_nh ** 6 - 11.852 * F_nh ** 5 + 21.499 * F_nh ** 4 - 12.174 * F_nh ** 3 - 4.7873 * F_nh ** 2 + 5.8662 * F_nh - 0.2652
        if 2.25 <= h / T < 2.75:
            alpha_xx = 1.2205 * F_nh ** 6 - 5.4999 * F_nh ** 5 + 5.7966 * F_nh ** 4 + 6.6491 * F_nh ** 3 - 16.123 * F_nh ** 2 + 9.2016 * F_nh - 0.6342
        if 2.75 <= h / T < 3.25:
            alpha_xx = -0.4085 * F_nh ** 6 + 4.534 * F_nh ** 5 - 18.443 * F_nh ** 4 + 35.744 * F_nh ** 3 - 34.381 * F_nh ** 2 + 15.042 * F_nh - 1.3807
        if 3.25 <= h / T < 3.75:
            alpha_xx = 0.4078 * F_nh ** 6 - 0.919 * F_nh ** 5 - 3.8292 * F_nh ** 4 + 15.738 * F_nh ** 3 - 19.766 * F_nh ** 2 + 9.7466 * F_nh - 0.6409
        if 3.75 <= h / T < 4.5:
            alpha_xx = 0.3067 * F_nh ** 6 - 0.3404 * F_nh ** 5 - 5.0511 * F_nh ** 4 + 16.892 * F_nh ** 3 - 20.265 * F_nh ** 2 + 9.9002 * F_nh - 0.6712
        if 4.5 <= h / T < 5.5:
            alpha_xx = 0.3212 * F_nh ** 6 - 0.3559 * F_nh ** 5 - 5.1056 * F_nh ** 4 + 16.926 * F_nh ** 3 - 20.253 * F_nh ** 2 + 10.013 * F_nh - 0.7196
        if 5.5 <= h / T < 6.5:
            alpha_xx = 0.9252 * F_nh ** 6 - 4.2574 * F_nh ** 5 + 5.0363 * F_nh ** 4 + 3.3282 * F_nh ** 3 - 10.367 * F_nh ** 2 + 6.3993 * F_nh - 0.2074
        if 6.5 <= h / T < 7.5:
            alpha_xx = 0.8442 * F_nh ** 6 - 4.0261 * F_nh ** 5 + 5.313 * F_nh ** 4 + 1.6442 * F_nh ** 3 - 8.1848 * F_nh ** 2 + 5.3209 * F_nh - 0.0267
        if 7.5 <= h / T < 8.5:
            alpha_xx = 0.1211 * F_nh ** 6 + 0.628 * F_nh ** 5 - 6.5106 * F_nh ** 4 + 16.7 * F_nh ** 3 - 18.267 * F_nh ** 2 + 8.7077 * F_nh - 0.4745

        if 8.5 <= h / T < 9.5:
            if F_nh < 0.6:
                alpha_xx = 1
            if F_nh >= 0.6:
                alpha_xx = -6.4069 * F_nh ** 6 + 47.308 * F_nh ** 5 - 141.93 * F_nh ** 4 + 220.23 * F_nh ** 3 - 185.05 * F_nh ** 2 + 79.25 * F_nh - 12.484
        if h / T >= 9.5:
            if F_nh < 0.6:
                alpha_xx = 1
            if F_nh >= 0.6:
                alpha_xx = -6.0727 * F_nh ** 6 + 44.97 * F_nh ** 5 - 135.21 * F_nh ** 4 + 210.13 * F_nh ** 3 - 176.72 * F_nh ** 2 + 75.728 * F_nh - 11.893

    V_2 = V_0 / alpha_xx

    #################################################################################################################


    """4) Wave resistance
    - 4th resistance component defined by Holtrop and Mennen (1982)
    - When the speed or the vessel size increases, the wave making resistance increases
    - In shallow water, the wave resistance shows an asymptotical behaviour by reaching the critical speed"""


    F_n = V_2 / np.sqrt(g * L)  # Froude number

    # parameter c_7 is determined by the B/L ratio
    if B / L < 0.11:
        c_7 = 0.229577 * (B / L) ** 0.33333
    if B / L > 0.25:
        c_7 = 0.5 - 0.0625 * (L / B)
    else:
        c_7 = B / L

    # half angle of entrance in degrees
    i_E = 1 + 89 * np.exp(-((L / B) ** 0.80856) * ((1 - C_wp) ** 0.30484) * (
                (1 - C_p - 0.0225 * lcb) ** 0.6367) * ((L_R / B) ** 0.34574) * (
                                           (100 * delta / (L ** 3)) ** 0.16302))

    c_1 = 2223105 * (c_7 ** 3.78613) * ((T / B) ** 1.07961) * (90 - i_E) ** (-1.37165)
    c_2 = 1  # accounts for the effect of the bulbous bow, which is not present at inland ships
    c_5 = 1 - (0.8 * A_T) / (
                B * T * C_M)  # influence of the transom stern on the wave resistance

    # parameter c_15 depoends on the ratio L^3 / delta
    if (L ** 3) / delta < 512:
        c_15 = -1.69385
    if (L ** 3) / delta > 1727:
        c_15 = 0
    else:
        c_15 = -1.69385 + (L / (delta ** (1 / 3)) - 8) / 2.36

    # parameter c_16 depends on C_p
    if C_p < 0.8:
        c_16 = 8.07981 * C_p - 13.8673 * (C_p ** 2) + 6.984388 * (C_p ** 3)
    else:
        c_16 = 1.73014 - 0.7067

    m_1 = 0.0140407 * (L / T) - 1.75254 * ((delta) ** (1 / 3) / L) - 4.79323 * (
                B / L) - c_16

    m_4 = 0.4 * c_15 * np.exp(-0.034 * (F_n ** (-3.29)))

    if L / B < 12:
        lmbda = 1.446 * C_p - 0.03 * (L / B)
    else:
        lmbda = 1.446 * C_p - 0.036

    # parameters needed for RW_2
    c_17 = 6919.3 * (C_M ** (-1.3346)) * ((delta / (L ** 3)) ** 2.00977) * (
                (L / B - 2) ** 1.40692)
    m_3 = -7.2035 * ((B / L) ** 0.326869) * ((T / B) ** 0.605375)

    ######### When Fn < 0.4
    RW_1 = c_1 * c_2 * c_5 * delta * rho * g * np.exp(
        m_1 * (F_n ** (-0.9)) + m_4 * np.cos(lmbda * (F_n ** (-2))))

    ######## When Fn > 0.5
    RW_2 = c_17 * c_2 * c_5 * delta * rho * g * np.exp(
        m_3 * (F_n ** (-0.9)) + m_4 * np.cos(lmbda * (F_n ** (-2))))

    if F_n < 0.4:
        R_W = RW_1 / 1000  # kN
    if F_n > 0.55:
        R_W = RW_2 / 1000  # kN
    else:
        R_W = (RW_1 + ((10 * F_n - 4) * (RW_2 - RW_1)) / 1.5) / 1000  # kN

    #################################################################################################################


    """5) Residual resistance terms
    - Holtrop and Mennen (1982) defined three residual resistance terms:
    - 1) Resistance due to the bulbous bow (not incorporated since inland ships in general don't have a bulb)
    - 2) Resistance due to immersed transom
    - 3) Resistance due to model-ship correlation """

    # Resistance due to immersed transom: R_TR [kN]
    F_nt = V_2 / np.sqrt(
        2 * g * A_T / (B + B * C_wp))  # Froude number based on transom immersion
    c_6 = 0.2 * (1 - 0.2 * F_nt)  # Assuming F_nt < 5, this is the expression for coefficient c_6

    R_TR = (0.5 * rho * (V_2 ** 2) * A_T * c_6) / 1000

    # Model-ship correlation resistance: R_A [kN]

    if T / L < 0.04:
        c_4 = T / L
    else:
        c_4 = 0.04
    c_2 = 1

    C_A = 0.006 * (L + 100) ** (-0.16) - 0.00205 + 0.003 * np.sqrt(L / 7.5) * (
                C_b ** 4) * c_2 * (0.04 - c_4)

    ####### Holtrop and Mennen in the document of Sarris, 2003 #######
    R_A = (0.5 * rho * (V_2 ** 2) * S_T * C_A) / 1000  # kW

    #################################################################################################################

    R_tot = R_f*one_k1 + R_APP + R_W + R_TR + R_A
    print('total resistance =',R_tot)
    P_empty=R_tot * V_0
    P_full = R_tot * 1.2 * V_0
    return P_empty

In [3]:
class HasEnergyPlugin:
    """Mixin for Activity to initialize WeatherPluginActivity."""
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        energy_plugin = EnergyPlugin()
        self.register_plugin(plugin=energy_plugin, priority=3)


class EnergyPlugin(model.AbstractPluginClass):
    """Mixin for all activities to add delay and downtime."""

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
        
    def post_process(
        self, env, activity_log, activity, start_activity, *args, **kwargs
    ):
        print('post_process',activity.mover,activity)
        V_0=activity.mover.compute_v(None)
        self.power = calculate_power(V_0=V_0)

        activity_duration = env.now - start_activity
        activity.log_entry(
            t=env.now,
            activity_id=activity.id,
            activity_state=core.LogState.UNKNOWN,
            additional_state={
                "power":self.power,
                "activity_duration":activity_duration,
                "energy_used":self.power*activity_duration,
            }
        )
        return {}


#### 1. Initialise simpy environment

In [4]:
# setup environment (simulation time needs to match the available weather data)
simulation_start = 0
my_env = simpy.Environment(initial_time=simulation_start)


#### 2. Define object classes

In [5]:
# create a Site object based on desired mixin classes
Site = type(
    "Site",
    (
        core.Identifiable,
        core.Log,
        core.Locatable,
        core.HasContainer,
        core.HasResource,
    ),
    {},
)

# create a TransportProcessingResource object based on desired mixin classes
TransportProcessingResource = type(
    "TransportProcessingResource",
    (
        core.Identifiable,
        core.Log,
        core.ContainerDependentMovable,
        core.Processor,
        core.HasResource,
        core.LoadingFunction,
        core.UnloadingFunction
    ),
    {},
)

#### 3. Create objects
##### 3.1. Create site object(s)

In [6]:
# prepare input data for from_site
location_from_site = shapely.geometry.Point(4.0, 52.11428333)
data_from_site = {"env": my_env,
                  "name": "from_site",
                  "geometry": location_from_site,
                  "capacity": 12,
                  "level": 12
                 }
# instantiate from_site 
from_site = Site(**data_from_site)

# prepare input data for to_site
location_to_site = shapely.geometry.Point(4.1, 52.11428333)
data_to_site = {"env": my_env,
                "name": "to_site",
                "geometry": location_to_site,
                "capacity": 12,
                "level": 0
               }
# instantiate to_site 
to_site = Site(**data_to_site)

##### 3.2. Create vessel object(s)

In [7]:
# prepare input data for vessel_01
data_vessel01 = {"env": my_env,
                 "name": "vessel01",
                 "geometry": location_from_site, 
                 "loading_rate": 1,
                 "unloading_rate": 1,
                 "capacity": 4,
                 "compute_v": lambda x: 5
               }
# instantiate vessel_01 
vessel01 = TransportProcessingResource(**data_vessel01)
# Monkeypatch with L: Data from EIS HAM318
# vessel01.L = 225.71
# vessel01.B = 32.046
# vessel01.T = 13.55

##### 3.3 Create activity/activities

In [8]:
# initialise registry
registry = {}
keep_resources = {}

In [9]:
# create delay sequence activities
DelaySequenceActivity =  type(
    "TestShiftActivity",
    (
        model.SequentialActivity,  # the order is critical!
    ),
    {},
)

DelayWhileActivity =  type(
    "TestShiftActivity",
    (
        model.WhileActivity,  # the order is critical!
    ),
    {},
)

DelayMoveActivity =  type(
    "TestMoveActivity",
    (
        HasEnergyPlugin,
        model.MoveActivity,  # the order is critical!
    ),
    {},
)

DelayShiftActivity =  type(
    "TestShiftActivity",
    (
        model.ShiftAmountActivity,  # the order is critical!
    ),
    {},
)

EnergyBasicActivity =  type(
    "TestShiftActivity",
    (
        model.BasicActivity,  # the order is critical!
    ),
    {},
)

In [10]:
# create a list of the sub processes
sub_processes = [
    DelayMoveActivity(
        env=my_env,
        name="sailing empty",
        registry=registry,
        mover=vessel01,
        destination=from_site,
    ),
    DelayShiftActivity(
        env=my_env,
        name="load cargo",
        registry=registry,
        processor=vessel01,
        origin=from_site,
        destination=vessel01,
        amount=4,
        duration=0,
    ),
    DelayMoveActivity(
        env=my_env,
        name="sailing full",
        registry=registry,
        mover=vessel01,
        destination=to_site,
    ),
    DelayShiftActivity(
        env=my_env,
        name="unload cargo",
        registry=registry,
        processor=vessel01,
        origin=vessel01,
        destination=to_site,
        amount=4,
        duration=0,
    ),
    EnergyBasicActivity(
        env=my_env,
        name="Basic activity",
        registry=registry,
        duration=0,
        additional_logs=[vessel01],
    ),
]

# create a 'sequential activity' that is made up of the 'sub_processes'
sequential_activity = DelaySequenceActivity(
    env=my_env,
    name="Single run process",
    registry=registry,
    sub_processes=sub_processes,
)

# create a while activity that executes the 'sequential activity' while a stop condition is not met 
while_activity = DelayWhileActivity(
    env=my_env,
    name="While activity",
    registry=registry,
    sub_processes=[sequential_activity],
    condition_event=[{"type": "container", "concept": to_site, "state": "full"}],
)

#### 4. Register processes and run simpy

In [11]:
model.register_processes([while_activity])
my_env.run()

post_process <__main__.TransportProcessingResource object at 0x000001B0BA01EE60> <__main__.TestMoveActivity object at 0x000001B0CD02F460>
total resistance = 515.8799993865017
post_process <__main__.TransportProcessingResource object at 0x000001B0BA01EE60> <__main__.TestMoveActivity object at 0x000001B0CD02FEE0>
total resistance = 515.8799993865017
post_process <__main__.TransportProcessingResource object at 0x000001B0BA01EE60> <__main__.TestMoveActivity object at 0x000001B0CD02F460>
total resistance = 515.8799993865017
post_process <__main__.TransportProcessingResource object at 0x000001B0BA01EE60> <__main__.TestMoveActivity object at 0x000001B0CD02FEE0>
total resistance = 515.8799993865017
post_process <__main__.TransportProcessingResource object at 0x000001B0BA01EE60> <__main__.TestMoveActivity object at 0x000001B0CD02F460>
total resistance = 515.8799993865017
post_process <__main__.TransportProcessingResource object at 0x000001B0BA01EE60> <__main__.TestMoveActivity object at 0x00000

#### 5. Inspect results
##### 5.1 Inspect logs

In [12]:
df = pd.concat(
    [
        plot.get_log_dataframe(act, [while_activity, sequential_activity, *sub_processes])
        for act in sub_processes
    ]
).sort_values(by=['Timestamp'])
df

,Activity,Timestamp,ActivityState,power,activity_duration,energy_used
0,sailing empty,1970-01-01 00:00:00.000000,START,NaN,NaN,NaN
1,sailing empty,1970-01-01 00:00:00.000000,STOP,NaN,NaN,NaN
2,sailing empty,1970-01-01 00:00:00.000000,UNKNOWN,2579.399997,0.000000,0.000000e+00
0,sailing full,1970-01-01 00:00:00.000000,START,NaN,NaN,NaN
0,load cargo,1970-01-01 00:00:00.000000,START,NaN,NaN,NaN
1,load cargo,1970-01-01 00:00:00.000000,STOP,NaN,NaN,NaN
1,Basic activity,1970-01-01 00:22:50.059695,STOP,NaN,NaN,NaN
0,Basic activity,1970-01-01 00:22:50.059695,START,NaN,NaN,NaN
1,unload cargo,1970-01-01 00:22:50.059695,STOP,NaN,NaN,NaN
0,unload cargo,1970-01-01 00:22:50.059695,START,NaN,NaN,NaN


In [13]:
df['energy_used'].sum()

17669659.86372888

##### 5.2 Visualise gantt charts

In [14]:
objects = [while_activity, sequential_activity, vessel01, from_site, to_site]
objects.extend(sub_processes)
plot.get_gantt_chart(objects)